Instalamos todas las librerías requeridas:

In [1]:
!pip install seaborn fastapi scikit-learn uvicorn[standard] joblib nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.1/453.1 kB 31.8 MB/s eta 0:00:00


**Cargamos un dataset diferente de seaborn y Preparamos los datos:** Cargamos el dataset, eliminando valores nulos y seleccionando las columnas que se usarán como variables predictoras (features) y como la variable objetivo (target).

In [2]:
import seaborn as sns
import pandas as pd

# Cargar el dataset penguins
penguins = sns.load_dataset('penguins')
print(penguins.head())

# Eliminar filas con valores nulos
penguins = penguins.dropna()

# Seleccionar características numéricas y variable objetivo
X = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
y = penguins['species']

  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  
0       3750.0    Male  
1       3800.0  Female  
2       3250.0  Female  
3          NaN     NaN  
4       3450.0  Female  


**Entrenamiento de guardado del modelo:** Divicion de los datos en conjuntos de entrenamiento y prueba, entrenamiento de un modelo RandomForest y guardardo en disco para su uso posterior en la API.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Guardar el modelo
joblib.dump(model, 'penguins_model.joblib')
print("Modelo entrenado y guardado como 'penguins_model.joblib'")

Modelo entrenado y guardado como 'penguins_model.joblib'


**Creación de la aplicación fastapi:** Definimos la API que recibirá los datos de entrada, cargará el modelo y devolverá la predicción.

In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib

app = FastAPI()

class PenguinData(BaseModel):
    bill_length_mm: float
    bill_depth_mm: float
    flipper_length_mm: float
    body_mass_g: float

model = joblib.load('penguins_model.joblib')

@app.post('/predict')
async def predict(data: PenguinData):
    input_data = [[
        data.bill_length_mm,
        data.bill_depth_mm,
        data.flipper_length_mm,
        data.body_mass_g
    ]]
    pred = model.predict(input_data)[0]
    return {"prediction": pred}

**Probamos la aplicación fastapi:**

In [5]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()

# Ejecutar el servidor (esto permite probarlo dentro de Colab)
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [260]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:43016 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [260]


Prueba en la Terminal:
curl -X POST "http://localhost:8000/predict" \
  -H "Content-Type: application/json" \
  -d '{"bill_length_mm": 44.0, "bill_depth_mm": 17.0, "flipper_length_mm": 210.0, "body_mass_g": 4200.0}'

**Resumen final**
- Elegimos el dataset `penguins` de seaborn, adecuado para clasificación.
- Preparamos los datos seleccionando sólo las columnas numéricas y eliminando valores nulos.
- Entrenamos un modelo RandomForestClassifier para predecir la especie de pingüino.
- Guardamos el modelo entrenado.
- Creamos una API usando FastAPI que recibe los datos del pingüino y devuelve la predicción de la especie.
- Probamos la API enviando datos de ejemplo y recibiendo la predicción correspondiente.